In [1]:
from azureml.core import Workspace, Experiment
#change myworspace name accordingly
ws = Workspace.get(name="myworkspace")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A3KX7HH38 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: myworkspace
Azure region: southcentralus
Subscription id: 26fa4a48-44da-4329-be20-b2ed37dcaadc
Resource group: sdk


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from sklearn.linear_model import LogisticRegression
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({
    "C": uniform(1, 5),
    "max_iter": choice(5, 10, 20),
    }
)
    
# Specify a Policy
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

### YOUR CODE HERE ###
# Create a SKLearn estimator for use with train.py
est = SKLearn(
                source_directory='training',
                compute_target=compute_target,
                entry_script='train.py'

            )

# Create a HyperDriveConfig using the estimator, 
#hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                             estimator=est,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=12
                             max_concurrent_runs=4)

In [11]:
from azureml.core.run import Run
run_logger = Run.get_context()
run_logger.log("accuracy", float(val_accuracy))

NameError: name 'val_accuracy' is not defined

In [10]:
# Submit your hyperdrive run to the experiment and show run details 
#with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


RunId: HD_2455c351-09d7-4b8e-b15f-8a56bce7f2ea
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_2455c351-09d7-4b8e-b15f-8a56bce7f2ea?wsid=/subscriptions/26fa4a48-44da-4329-be20-b2ed37dcaadc/resourcegroups/sdk/workspaces/myworkspace

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-18T10:04:37.648752][API][INFO]Experiment created<END>\n""<START>[2020-12-18T10:04:38.069279][GENERATOR][INFO]Trying to sample '100' jobs from the hyperparameter space<END>\n"<START>[2020-12-18T10:04:38.7946112Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-18T10:04:39.172918][GENERATOR][INFO]Successfully sampled '100' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_2455c351-09d7-4b8e-b15f-8a56bce7f2ea
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_2455c351-09d7-4b8e-b15f-8a56bce7f2ea?wsid=/subscriptions/26fa4a48-44da-4329-be20-b2ed37d

{'runId': 'HD_2455c351-09d7-4b8e-b15f-8a56bce7f2ea',
 'target': 'cpu-cluster',
 'status': 'Canceled',
 'startTimeUtc': '2020-12-18T10:04:37.379791Z',
 'endTimeUtc': '2020-12-18T10:18:36.988518Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": "ac

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
### YOUR CODE HERE ###